In [7]:
import cv2
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
from pathlib import Path
from os import listdir
import concurrent.futures
from itertools import repeat
import time

In [3]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=2.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

def convert_from_cowc_to_planet(image,alpha = 1.2,beta = -50,small_to_large_image_size_ratio = 0.33,factor=1.3):
    '''convert an image from cowc dataset to an image similar to those made by planet satellite.
    the original resolution is 15cm, the target resolution is 50cm.
    Emulate pansharpening using laplacian unsharp mask, change contrast and brightness.
    The optimal values where found by eye, a better solution would automate this step.
    In output image a car is 40-50 pixels wide'''
    #params
    #contrast and brightness
    #alpha = 1.2 # Contrast control (1.0-3.0)
    #beta = -50 # Brightness control (0-100)
    #small_to_large_image_size_ratio = 0.33 #size ratio, that is how much I want to resize the original image
    #factor=1.3#how much to enlarge resized image, used to match cars width in pixels, in order for the cnn to perform better on test data
    
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    small_sharpened_img = cv2.resize(unsharp_mask(image), # original image
                           (0,0), # set fx and fy, not the final size
                           fx=small_to_large_image_size_ratio, 
                           fy=small_to_large_image_size_ratio, 
                           interpolation=cv2.INTER_NEAREST)

    small_sharpened_img = cv2.resize(small_sharpened_img, # original image
                           (0,0), # set fx and fy, not the final size
                           fx=1/small_to_large_image_size_ratio * factor, 
                           fy=1/small_to_large_image_size_ratio * factor)
    return small_sharpened_img

def convert_from_path_and_save(input_path,dest_path,img,**kwargs):
    img_path = input_path/img
    dest_path = dest_path/img
    input_img = cv2.imread(str(img_path))
    cv.imwrite(str(dest_path),convert_from_cowc_to_planet(input_img))

In [9]:
input_path = Path("../storage/data/Potsdam_ISPRS/all_data")
output_path = Path("../storage/data/Potsdam_ISPRS/all_data_transformed")
imgs = listdir(input_path)

In [11]:

#serial version
start = time.time()

for img in imgs:
    convert_from_path_and_save(input_path,output_path, img)
    
end = time.time()
print("Seconds elapsed for " +str(len(imgs)) + " images : " + str(end - start))

Seconds elapsed for 1000 images : 11.770115375518799


In [10]:
%time
#parallel version (need ssd to improve performance because I/O bounded problem)

start = time.time()
max_workers=8
chunksize = int(len(imgs)/max_workers)
with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
    executor.map(convert_from_path_and_save, repeat(input_path), repeat(output_path), imgs, chunksize=chunksize)

            
end = time.time()
print("Seconds elapsed for " +str(len(imgs)) + " images, parallel version : " + str(end - start))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
Seconds elapsed for 12800 images, parallel version : 188.88276982307434
